In [29]:
# загружаю признаки
import pandas as pd
features = pd.read_csv('features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [30]:
# удаляю признаки, которые заглядывают в будущее и которых нет в тестовой выборке
features.drop(columns=['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], inplace = True)

In [31]:
# функция поиска пропусков в признаках
def missingdata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
   
    return ms

In [32]:
# признаки с пропусками
missingdata(features)

,Total,Percent
first_blood_player2,43987,45.240152
radiant_flying_courier_time,27479,28.261853
dire_flying_courier_time,26098,26.841510
first_blood_time,19553,20.110048
first_blood_team,19553,20.110048
first_blood_player1,19553,20.110048
dire_bottle_time,16143,16.602900
radiant_bottle_time,15691,16.138023
radiant_first_ward_time,1836,1.888306
dire_first_ward_time,1826,1.878021


In [33]:
# missingdata(features) транспонирую, заполняю пропуски нулями
for (columnName, columnData) in (missingdata(features).T).iteritems():
    features[columnName].fillna(0, inplace = True)

In [34]:
# целевая переменная записана в поле radiant_win в последнем столбце, достаю признаки и значения
data_x = features.iloc[:, 0:(len(features.columns)-1)]
data_y = features.iloc[:, (len(features.columns)-1):(len(features.columns))]

In [35]:
# KFold
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
import time
import datetime

start_time = datetime.datetime.now()

clf = GradientBoostingClassifier(n_estimators=30, verbose=False, random_state=241, learning_rate = 0.2)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
f = np.mean(cross_val_score(clf, data_x, data_y.values.ravel(), cv=cv, scoring='roc_auc'))

print(f)
print('Time elapsed:', datetime.datetime.now() - start_time)

0.6981736838791232
Time elapsed: 0:05:39.555310


In [36]:
# попробую еще train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

start_time = datetime.datetime.now()

x_train, x_test, y_train, y_test=train_test_split(data_x, data_y, shuffle= True, test_size=0.2, random_state=241)
clf = GradientBoostingClassifier(n_estimators=30, verbose=False, random_state=241, learning_rate = 0.2)
clf.fit(x_train, y_train.values.ravel())
pred = clf.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, pred)

print(roc_auc_score(y_test, pred))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7040970790725902
Time elapsed: 0:01:19.014451


In [20]:
# train_test_split работает быстрее и точность показывает выше

In [24]:
# потом я варьировал различные значения параметров n_estimators, learning_rate и max_depth
# вот какое маскимальное значение точности я получил
start_time = datetime.datetime.now()

x_train, x_test, y_train, y_test=train_test_split(data_x, data_y, shuffle= True, test_size=0.2, random_state=241)
clf = GradientBoostingClassifier(n_estimators=1200, verbose=False, random_state=241, learning_rate = 0.2)
clf.fit(x_train, y_train.values.ravel())
pred = clf.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, pred)

print(roc_auc_score(y_test, pred))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7222778312535666
Time elapsed: 0:27:49.595623


In [27]:
test_data = pd.read_csv('features_test.csv', index_col='match_id')
for (columnName, columnData) in (missingdata(test_data).T).iteritems():
    test_data[columnName].fillna(0, inplace = True)